In [201]:
import pandas as pd
import pulp
from pulp import GLPK
import numpy as np

In [202]:
raw_df = pd.read_excel("/Users/yanhanjun/Documents/Envionment_.xlsx")
#raw_df.columns = raw_df.iloc[2]
raw_df = raw_df.drop(raw_df.columns[:2], axis=1)
raw_df = raw_df.drop(index=[0, 1])
raw_df.columns = raw_df.iloc[0]
raw_df.reset_index(drop=True, inplace=True)
raw_df = raw_df.drop(index=[0])
raw_df = raw_df[~raw_df['Company'].isna()]
raw_df.reset_index(drop=True, inplace=True)
raw_df.rename(columns={'2':'Index'})

2,Company,Year,Scope_1_GHG,Scope_2_GHG,Scope_3_GHG,Carbon_Footprint,GHG_Intensity,NonRenewable_Energy_Consumption_and_Production,Renewable_Energy_Use,Total_Energy_Use,...,Eneregy_by_NonRenewable,Coal,Natural_Gas,Crudel_Oil,Water_Intensity_Per_Sales,Water_recycled_Ratio,NonRecycled_Waste_ration,Waste_Recycled,Total_waste,Percentage_waste_Recycled
0,SONY US Equity,2023,1.77,5.66,162.45,169.87,283.35,89.9,758.95,7500,...,44356.9,0,44356.6,11.3,0,0,3.3,53.15,54.95,96.7
1,SONY US Equity,2022,1.45,6.74,124.93,133.12,255.6,95.1,356.68,7222.22,...,45152.45,0,45142.5,9.95,NaN,NaN,2,50.56,51.62,98
2,SONY US Equity,2021,1.53,6.74,146.44,154.71,283.36,94.4,165.64,2967.53,...,22505.5,0,22477.5,28,NaN,NaN,2,50.36,51.36,98.1
3,SONY US Equity,2020,2.15,13.61,170.3,186.06,237.57,95.8,116,2777.15,...,22497.82,0,22477.9,9.92,NaN,NaN,2.1,47.2,48.19,98
4,SONY US Equity,2019,2.97,18.68,260.18,281.83,263.1,98.1,52.8,2752.22,...,21546.3,0,21534.2,12.1,NaN,NaN,4.6,46.57,48.4,95.4
5,AAPl US Equity,2023,0.02,0,6.07,6.09,44.65,16.8,3489,4195.32,...,31349.53,0,31336,13.53,16.96,9.4,44.7,37.09,67.1,55.3
6,AAPl US Equity,2022,0.02,0,8.6,8.62,61.92,7.7,3245.68,3516.98,...,18922.61,0,18915.5,7.11,15.9,9.3,36.9,35.65,56.5,63.1
7,AAPl US Equity,2021,0.03,0,11.44,11.46,81.48,14.1,2854,3324.06,...,20363.05,0,20357.5,5.55,17.4,10,36.5,33.33,52.48,63.5
8,AAPl US Equity,2020,0.03,0,13.96,13.99,99.39,14.6,2580,3019.17,...,20295.98,0,20292.4,2.58,19.83,8.8,36.7,28.94,45.71,63.3
9,AAPl US Equity,2019,0.05,0,25.3,25.35,108.54,15.9,2430,2889.68,...,20294.91,0,20290.4,4.51,21.19,8.2,43.8,32.81,58.4,56.2


In [203]:
df = raw_df.copy()

In [204]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 22 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   Company                                         25 non-null     object
 1   Year                                            25 non-null     object
 2   Scope_1_GHG                                     25 non-null     object
 3   Scope_2_GHG                                     25 non-null     object
 4   Scope_3_GHG                                     25 non-null     object
 5   Carbon_Footprint                                25 non-null     object
 6   GHG_Intensity                                   25 non-null     object
 7   NonRenewable_Energy_Consumption_and_Production  24 non-null     object
 8   Renewable_Energy_Use                            24 non-null     object
 9   Total_Energy_Use                                24 non-n

In [205]:
df[['GHG_Intensity', 'Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production', 'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']] = \
    df[['GHG_Intensity', 'Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production', 'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']].astype('float64')

In [206]:
df_input = df[['Company', 'Year', 'GHG_Intensity', 'Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production', 'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']].interpolate(method='linear')

/var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/ipykernel_26348/1554244951.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_input = df[['Company', 'Year', 'GHG_Intensity', 'Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production', 'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']].interpolate(method='linear')


In [207]:
df_input.isna().sum()

2
Company                                           0
Year                                              0
GHG_Intensity                                     0
Energy_Intensity_per_Sales                        0
NonRenewable_Energy_Consumption_and_Production    0
Water_Intensity_Per_Sales                         0
NonRecycled_Waste_ration                          0
dtype: int64

In [208]:
df_input.head()

2,Company,Year,GHG_Intensity,Energy_Intensity_per_Sales,NonRenewable_Energy_Consumption_and_Production,Water_Intensity_Per_Sales,NonRecycled_Waste_ration
0,SONY US Equity,2023,283.35,96.22,89.9,0.000,3.3
1,SONY US Equity,2022,255.60,95.00,95.1,3.392,2.0
2,SONY US Equity,2021,283.36,40.75,94.4,6.784,2.0
3,SONY US Equity,2020,237.57,40.61,95.8,10.176,2.1
4,SONY US Equity,2019,263.10,40.76,98.1,13.568,4.6


In [209]:
raw_df_2 = pd.read_excel("/Users/yanhanjun/Documents/Envionment_.xlsx", sheet_name="ESG_Score")
raw_df_2 = raw_df_2.drop(raw_df_2.columns[:2], axis=1)
raw_df_2 = raw_df_2.drop(index=[0, 1])
raw_df_2.columns = raw_df_2.iloc[0]
raw_df_2.reset_index(drop=True, inplace=True)
raw_df_2 = raw_df_2.drop(index=[0])
raw_df_2 = raw_df_2[~raw_df_2['Company'].isna()]
raw_df_2.reset_index(drop=True, inplace=True)
raw_df_2.head()

2,Company,Year,ESG_Score,GHG_1,GHG_2,GHG_3,1_plus_2,1_plus_2_plus_3
0,SONY US Equity,2023,5.09,2.72,8.73,250.54,11.45,261.99
1,SONY US Equity,2022,5.03,2.63,12.28,227.68,14.92,242.6
2,SONY US Equity,2021,4.97,2.36,10.44,226.74,12.8,239.54
3,SONY US Equity,2020,4.79,2.41,15.22,190.47,17.63,208.09
4,SONY US Equity,2019,4.51,2.48,15.58,216.95,18.05,235


In [210]:
raw_df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Company          25 non-null     object
 1   Year             25 non-null     object
 2   ESG_Score        25 non-null     object
 3   GHG_1            24 non-null     object
 4   GHG_2            24 non-null     object
 5   GHG_3            24 non-null     object
 6   1_plus_2         24 non-null     object
 7   1_plus_2_plus_3  24 non-null     object
dtypes: object(8)
memory usage: 1.7+ KB


In [211]:
df_output = raw_df_2.drop(columns=['Company', 'Year'])
df_output = df_output.astype(float)
df_output = df_output.interpolate(method='linear')

In [212]:
df_concat = pd.concat([df_input, df_output], axis=1)

In [213]:
df_concat.shape

(25, 13)

In [214]:
df_concat.columns

Index(['Company', 'Year', 'GHG_Intensity', 'Energy_Intensity_per_Sales',
       'NonRenewable_Energy_Consumption_and_Production',
       'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration', 'ESG_Score',
       'GHG_1', 'GHG_2', 'GHG_3', '1_plus_2', '1_plus_2_plus_3'],
      dtype='object', name=2)

In [215]:
df_concat['Year'] = df_concat['Year'].astype(int)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 13 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Company                                         25 non-null     object 
 1   Year                                            25 non-null     int64  
 2   GHG_Intensity                                   25 non-null     float64
 3   Energy_Intensity_per_Sales                      25 non-null     float64
 4   NonRenewable_Energy_Consumption_and_Production  25 non-null     float64
 5   Water_Intensity_Per_Sales                       25 non-null     float64
 6   NonRecycled_Waste_ration                        25 non-null     float64
 7   ESG_Score                                       25 non-null     float64
 8   GHG_1                                           25 non-null     float64
 9   GHG_2                                        

In [ ]:
df_concat.Year.unique()


array([2023, 2022, 2021, 2020, 2019])

In [239]:
df_concat.to_csv("DEA_data.csv")

In [230]:
Input_DATAFRAME = df_concat[['Company', 'Year', 'Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                      'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']]
Output_DATAFRAME = df_concat[['Company', 'Year', 'GHG_Intensity', 'ESG_Score']]
Input_DATAFRAME

2,Company,Year,Energy_Intensity_per_Sales,NonRenewable_Energy_Consumption_and_Production,Water_Intensity_Per_Sales,NonRecycled_Waste_ration
0,SONY US Equity,2023,96.22,89.90,0.000,3.300
1,SONY US Equity,2022,95.00,95.10,3.392,2.000
2,SONY US Equity,2021,40.75,94.40,6.784,2.000
3,SONY US Equity,2020,40.61,95.80,10.176,2.100
4,SONY US Equity,2019,40.76,98.10,13.568,4.600
5,AAPl US Equity,2023,11.68,16.80,16.960,44.700
6,AAPl US Equity,2022,9.67,7.70,15.900,36.900
7,AAPl US Equity,2021,10.86,14.10,17.400,36.500
8,AAPl US Equity,2020,12.30,14.60,19.830,36.700
9,AAPl US Equity,2019,12.53,15.90,21.190,43.800


In [234]:
inputs_t1 = np.array(Input_DATAFRAME[Input_DATAFRAME['Year'] == 2019][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                      'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])  # 时间1的输入
outputs_t1 = np.array(Output_DATAFRAME[Output_DATAFRAME['Year'] == 2019][['GHG_Intensity', 'ESG_Score']])
inputs_t2 = np.array(Input_DATAFRAME[Input_DATAFRAME['Year'] == 2020][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                      'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])  # 时间1的输入
outputs_t2 = np.array(Output_DATAFRAME[Output_DATAFRAME['Year'] == 2020][['GHG_Intensity', 'ESG_Score']])

In [235]:
num_dmus = inputs_t1.shape[0]
num_inputs = inputs_t1.shape[1]
num_outputs = outputs_t1.shape[1]

In [236]:
def calculate_efficiency(inputs, outputs, reference_inputs, reference_outputs):
    model = pulp.LpProblem("Efficiency_Calculation", pulp.LpMaximize)
    lambdas = [pulp.LpVariable(f"lambda_{j}", lowBound=0) for j in range(num_dmus)]
    model += pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(num_dmus)])
    for k in range(num_inputs):
        model += pulp.lpSum([lambdas[j] * reference_inputs[j, k] for j in range(num_dmus)]) <= inputs[i, k]
    model += pulp.lpSum(lambdas) == 1
    model.solve()
        
    if model.status == pulp.LpStatusOptimal:
        efficiency_score = pulp.value(pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(num_dmus)])) / outputs[i, 0]
    else:
        efficiency_score = None
        
    return efficiency_score

In [ ]:
malmquist_indices = []
for i in range(num_dmus):
    eff_t1_t1 = calculate_efficiency(inputs_t1, outputs_t1, inputs_t1, outputs_t1)
    eff_t1_t2 = calculate_efficiency(inputs_t1, outputs_t1, inputs_t2, outputs_t2)
    eff_t2_t1 = calculate_efficiency(inputs_t2, outputs_t2, inputs_t1, outputs_t1)
    eff_t2_t2 = calculate_efficiency(inputs_t2, outputs_t2, inputs_t2, outputs_t2)

    if all(v is not None for v in [eff_t1_t1, eff_t2_t2, eff_t1_t2, eff_t2_t1]):
        EC = eff_t2_t2 / eff_t1_t1 if eff_t1_t1 != 0 else None
        TC = np.sqrt((eff_t1_t2 * eff_t2_t2) / (eff_t1_t1 * eff_t2_t1)) if all(v != 0 for v in [eff_t1_t1, eff_t2_t1]) else None
        malmquist_index = EC * TC if EC is not None and TC is not None else None
        malmquist_indices.append(malmquist_index)
    else:
        malmquist_indices.append(None)

for i, index in enumerate(malmquist_indices):
    print(f"DMU {i + 1} Malmquist Index: {index}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/711391c7d6a1418e9fb348ea7f1254d0-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/711391c7d6a1418e9fb348ea7f1254d0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 5 rows, 5 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 5 (0) columns and 25 (0) elements
0  Obj -0 Primal inf 0.96715805 (1) Dual inf 3515.256 (5)
0  Obj -0 Primal inf 0.96715805 (1) Dual inf 1.3835202e+11 (5)
2  Obj 263.1
Optimal - objective value 263.1
Optimal objective 263.1 - 2 iterations time 0.002
Option for printingOptions change